<a href="https://colab.research.google.com/github/ehdghks316/itwill_studyGroup_D/blob/main/4_22%ED%8C%8C%EC%9D%B4%EC%8D%AC(Randomforest%2C).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
titanic = pd.read_csv('c:/data/titanic.csv')
titanic.info()
titanic.head()
titanic.info()
# 종속변수, 목표변수, 결과변수, 정답레이블 : survived 컬럼
titanic.survived.unique()
0  = 사망 , 1 = 생존

# 설명변수, 입력변수, 독립변수
    # 티켓등급
titanic.pclass.unique()
    # 성별
titanic.gender.unique()
    # 나이
titanic.age.describe()
    # 함께 탐승한 형제자매, 배우자 수
titanic.sibsp.describe()
    # 함께 탑승한 부모님, 자식수
titanic.parch.describe()
    # 티켓번호
titanic.ticket
    # 운임
titanic.fare.describe()
    # 객실번호
titanic.cabin
    # 탑승항구
titanic.embarked.unique()
S = Southampton, Q = Queenstown, C = Cherbourg

# 2개의 문자값을 수치형으로 바꾸는 작업(female->0 male->1)
1.
titanic.gender.map({'female':0,'male':1})

2.
from collections import Counter
from sklearn.preprocessing import LabelEncoder
gender_le = LabelEncoder()
titanic.gender = gender_le.fit_transform(titanic['gender'])
titanic.gender
Counter(titanic.gender)

# 열별로 null값의 수
titanic.isnull().sum()

# 특정 열의 null값 확인
titanic.age.isnull().sum()

titanic.age.describe()

# age 중앙값
titanic.age.median()

# age null값을 중앙값으로 수정
titanic.age.fillna(titanic.age.median(),inplace=True)
titanic.age.isnull().sum()

Counter(titanic.embarked)

# one-hot-encoding
     C  Q  S
0    0  0  1
1    1  0  0
2    0  0  1
3    0  0  1
4    0  0  1
pd.get_dummies(titanic.embarked,prefix='embarked') # one-hot-encoding 작업 수행, prefix=접두어
     embarked_C  embarked_Q  embarked_S
0             0           0           1
1             1           0           0
2             0           0           1
3             0           0           1
4             0           0           1
..          ...         ...         ...
886           0           0           1
887           0           0           1
888           0           0           1
889           1           0           0
890           0           1           0

embarked_dummies = pd.get_dummies(titanic.embarked,prefix='embarked')

titanic = pd.concat([titanic,embarked_dummies],axis=1) # concat : 열 or 행 단위로 붙이기
titanic.info()

feature_col = ['pclass','gender','age','embarked_S','embarked_Q']
x = titanic[feature_col] # 학습할 데이터
y = titanic.survived # 정답 데이터

from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(x,y)
model.classes_
z = pd.DataFrame({'features':feature_col,
              'importance' : model.feature_importances_})

# 시각화
import matplotlib.pyplot as plt
z.plot()

[문제204] 유방암 데이터를 의사결정나무를 이용해서 분류해주세요.
wisc = pd.read_csv('c:/data/wisc_bc_data.csv')
wisc.info()
wisc
# 정답레이블
wisc.diagnosis

# 정답레이블 수치화
wisc.diagnosis = wisc.diagnosis.map({'B':'Benign','M':'Malignant'})
Counter(wisc.diagnosis)

# null값 확인
wisc.isnull().sum()


x = wisc.iloc[:,2:]
y = wisc.iloc[:,1]

from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(x,y)

x.columns
model.feature_importances_
pd.DataFrame({'features':x.columns,
              'importance':model.feature_importances_})

#------------------------------------- 강사님
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

x = wisc.iloc[:,2:]
y = wisc.diagnosis
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

wisc_model = DecisionTreeClassifier()
wisc_model.fit(x_train,y_train)

wisc_model.score(x_train,y_train)
wisc_model.score(x_test,y_test)

y_pred = wisc_model.predidct(x_test)
accuracy_score(y_test,y_pred)

wisc_result = pd.DataFrame({'feature':x.columns,'importance':wisc_model.feature_importances_})
wisc_result.sort_values(by='importance',ascending=False)

#------------------------------
dtree = DecisionTreeClassifier()
parameters = {'max_depth':[5,6,7,8,9],'min_samples_split':[4,5,6,7,8,9,10]}
grid_tree = GridSearchCV(dtree,param_grid=parameters,cv=5,refit=True,return_train_score=True)
grid_tree.fit(x_train,y_train)
scores_df = pd.DataFrame(grid_tree.cv_results_)
scores_df[['params','mean_test_score','rank_test_score']]

print('최적 파라미터',grid_tree.best_params_)
print('최고 정확도',grid_tree.best_score_)

y_pred = grid_tree.predict(x_test)
accuracy_score(y_pred,y_test)

tree model = grid_tree.best_estimator_
accuracy_score(tree_model.predict(x_test),y__test)
tree_model.feature_importances_
wisc_result = pd.DataFrame({'feature':x.columns,'importance':tree_model.feature_importances_})
wisc_result.sort_values(by='importance',ascending=False)

from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test,y_pred))
print(classifiaction_report(y_test,y_pred))


▶RandomForest
- 매 실행시 마다 랜덤하게 관측치와 변수(컬럼)를 선택하므로 실행결과가 조금씩 달라지게 된다.
- decision tree와 bagging을 결합한 알고리즘
- 학습데이터에서 중복을(복원추출) 허용하여 랜덤샘플링을 통해서 의사결정나무모델을 구축
  학습데이터 수 만큼 샘플링한다. 배깅(bagging, bootstrap aggregating)
- 의사결정나무 모델 구축시 변수도 무작위 선택한다. 단 하나의 모델안에서 변수는 중복불가능
- 의사결정나무 모델 결과의 투표를 통해서 클래스를 선택한다.


iris = pd.read_csv('c:/data/iris.csv')
iris.info()
iris.sample(n=150,replace=True) # sample : 데이터에서 랜덤으로 추출, 150개를 복원추출

import random

col_idx = []

for i in range(2):
    idx = random.randint(0,3) # 0~3까지 랜덤하게 하나 뽑기
    while idx in col_idx:
        idx = random.randint(0,3)
    col_idx.append(idx)

col_idx

data = iris.sample(n=150,replace=True)
x_train = data[data.columns[col_idx]]
y_train = data.Name

iris_model = DecisionTreeClassifier()
iris_model.fit(x_train,y_train)
iris_model.score(x_train,y_train)

iris_result = pd.DataFrame({'feature':x_train.columns,'importance':iris_model.feature_importances_})
iris_result.sort_values(by='importance',ascending=False)

pred_data = [5.1,3.5,1.4,0.2]
pred_data[col_idx[0]]
pred_data[col_idx[1]]

q = []
q.append(iris_model.predict([[pred_data[col_idx[0]],pred_data[col_idx[1]]]])[0])
q

Counter(q).most_common(1)[0][0]


#--------------------------
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

wisc = pd.read_csv('c:/data/wisc_bc_data.csv')
wisc.info()
wisc.diagnosis = wisc.diagnosis.map({'B':'Benign','M':'Malignant'})
Counter(wisc.diagnosis)
x = wisc.iloc[:,2:]
y = wisc.diagnosis
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

from sklearn.ensemble import RandomForestClassifier
#oob_score(out-of-bag_score) 부트스트랩(랜덤하게 중복샘플링) 샘플링시 선택되지 않은 샘플
model_rf = RandomForestClassifier(n_estimators=1000,oob_score=True)
model_rf.fit(x_train,y_train)
model_rf.score(x_train,y_train)
model_rf.score(x_test,y_test) # decisiontree와의 차이 비교해보기
model_rf.oob_score_

confusion_matrix(y_test,model_rf.predict(x_test))


rftree = RandomForestClassifier()
parameters = {'max_depth':[5,6,7,8,9],'min_samples_split':[4,5,6,7,8,9,10],'max_features':[5,6,7,8,9,10],'n_estimators':[100,200,300,400,500]}
grid_tree = GridSearchCV(rftree,param_grid=parameters,cv=5,refit=True,return_train_score=True,n_jobs=-1) # n_jobs=-1 컴퓨터의 모든 cpu를 다 사용해서 처리하겠어( 다른 프로그램들은 느려질 수 있음)
grid_tree.fit(x_train,y_train) # 상당히 오래걸림
scores_df = pd.DataFrame(grid_tree.cv_results_)
scores_df[['params','mean_test_score','rank_test_score']]

print('최적 파라미터',grid_tree.best_params_)
print('최고 정확도',grid_tree.best_score_)

y_pred = grid_tree.predict(x_test)
accuracy_score(y_pred,y_test)

tree model = grid_tree.best_estimator_
accuracy_score(tree_model.predict(x_test),y__test)
tree_model.feature_importances_
wisc_result = pd.DataFrame({'feature':x.columns,'importance':tree_model.feature_importances_})
wisc_result.sort_values(by='importance',ascending=False)

from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test,y_pred))
print(classifiaction_report(y_test,y_pred))